<a href="https://colab.research.google.com/github/thspanos/AI-LAb/blob/main/IDS_example_Thomas_Spanos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#εγκατάσταση πακέτου tensorflow (hhttps://www.tensorflow.org/)
!pip install tensorflow
#εγκατάσταση πακέτου scikit-learn (https://scikit-learn.org/)
!pip install scikit-learn

In [2]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight

try:
    from models import models_ddos
except ModuleNotFoundError:
    models_ddos = None  # placeholder για να μην αποτύχει το import τώρα

In [3]:
epochs = 100
nclass = 3  # το dataset έχει 3 κλάσεις

In [4]:
def loadDataset():
    # Put dataset path here !
    filename = 'https://raw.githubusercontent.com/thspanos/AI-LAb/refs/heads/main/Data/pcap_data.csv'

    trainfile = pd.read_csv(filename)

    # --- labels από τη στήλη 'target' (όπως είναι στο dataset) ---
    label = trainfile['target'].astype('str')

    # --- Χρήση όλων των features εκτός του target (αντί του inx_sel του παλιού dataset) ---
    data = trainfile.drop(columns=['target']).to_numpy()

    # MIN-MAX normalization (όπως είχες)
    dmin = data.min(axis=0)
    dmax = data.max(axis=0)
    data = (data - dmin) / (dmax - dmin)
    # data = np.log(data - dmin + 1.0)

    # Test data 20%
    train_data, test_data, train_label, test_label = \
        train_test_split(data, label, test_size=0.20, stratify=label)

    # Train 70%, Validation 10%
    train_data, val_data, train_label, val_label = \
        train_test_split(train_data, train_label, test_size=0.125, stratify=train_label)

    return (train_data.astype('float32'), train_label.astype(str),
            val_data.astype('float32'),   val_label.astype(str),
            test_data.astype('float32'),  test_label.astype(str))

In [6]:
# -- LOAD DATA -----------------------------------------------------------------
train_data, train_labelp, val_data, val_labelp, test_data, test_labelp = loadDataset()

# labels -> int (π.χ. '2.0' -> 2)
train_labelp = pd.to_numeric(train_labelp, errors='coerce').astype('int32')
val_labelp   = pd.to_numeric(val_labelp,   errors='coerce').astype('int32')
test_labelp  = pd.to_numeric(test_labelp,  errors='coerce').astype('int32')

# to_categorical
train_label = to_categorical(train_labelp, nclass)
val_label   = to_categorical(val_labelp,   nclass)
test_label  = to_categorical(test_labelp,  nclass)

print('train_data.shape=', train_data.shape)
print('test_data.shape=',  test_data.shape)
print('test_data.shape=',  val_data.shape)

# get the number of features
inshape = train_data.shape[1]

# Class balancing weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labelp),
    y=train_labelp
)

class_weights = {i: class_weights[i] for i in range(len(class_weights))}

train_data.shape= (10476, 25)
test_data.shape= (2994, 25)
test_data.shape= (1497, 25)


In [9]:
os.makedirs('./savemodels', exist_ok=True)

earlyStopping = EarlyStopping(
    monitor='val_loss',
    patience=30,
    verbose=0,
    mode='min'
)

# val_acc -> val_accuracy (όνομα metric στα logs του TF/Keras)
modelCheckPoint = ModelCheckpoint(
    './savemodels/model5class.{epoch:03d}-{val_accuracy:.4f}.weights.h5',
    save_best_only=True,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max'
)

# reduce_lr_loss = ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.1,
#     patience=7,
#     verbose=1,
#     epsilon=1e-4,
#     mode='min'
# )

In [10]:
    # -- Baseline models-----------------------------------------------------------
# Αν δεν υπάρχει το τοπικό module `models_ddos`, ορίζουμε ελάχιστο fallback για να δουλέψει στο Colab
if models_ddos is None:
    import tensorflow as tf
    from tensorflow.keras import layers, models as kmodels

    def _model_conv1D(lr, N, inshape):
        # Δέχεται 2D input (batch, inshape) και το μετατρέπει εσωτερικά σε (batch, inshape, 1)
        inputs = tf.keras.Input(shape=(inshape,))
        x = layers.Lambda(lambda t: tf.expand_dims(t, axis=-1))(inputs)
        x = layers.Conv1D(N, kernel_size=3, padding='same', activation='relu')(x)
        x = layers.MaxPooling1D(pool_size=2)(x)
        x = layers.Conv1D(N, kernel_size=3, padding='same', activation='relu')(x)
        x = layers.GlobalAveragePooling1D()(x)
        x = layers.Dense(N, activation='relu')(x)
        outputs = layers.Dense(nclass, activation='softmax')(x)
        model = tf.keras.Model(inputs, outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(lr),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model

    class _ModelsShim:
        @staticmethod
        def model_conv1D(lr, N, inshape):
            return _model_conv1D(lr, N, inshape)

    models_ddos = _ModelsShim()

# -- Conv1d
model = models_ddos.model_conv1D(lr=1e-4, N=64, inshape=inshape)
# -- Dense
# model = models_ddos.model_dense(lr=1e-4, N=64, inshape=inshape)
# -- LSTM
# model = models_ddos.model_lstm(lr=1e-4, N=64, inshape=inshape)

model.summary()
# -----------------------------------------------------------------------------
# print model to an image file
# dot_img_file = 'model1.png'
# plot_model(model, to_file=dot_img_file, show_shapes=True)

IndentationError: expected an indented block after 'if' statement on line 3 (ipython-input-3093621710.py, line 4)

In [ ]:
# -- TRAIN MODEL --------------------------------------------------------------
history = model.fit(
    train_data,
    train_label,
    shuffle=True,
    epochs=epochs,
    batch_size=256,  # 256,#128,#32, 64
    # validation_data=validation_generator,
    # validation_split=0.2,
    # validation_data=(val_data,val_label),
    validation_data=(val_data, val_label),
    callbacks=[modelCheckPoint],
    class_weight=class_weights
)

In [ ]:
# -- Load best model ----------------------------------------------------------
# προσοχή: φιλτράρουμε μόνο .hdf5 και ελέγχουμε αν υπάρχουν αρχεία
ckpt_dir = './savemodels'
files = [f for f in os.listdir(ckpt_dir) if f.endswith('.weights.h5')]
files = np.sort(files)
if files.size == 0:
    raise FileNotFoundError("Δεν βρέθηκαν checkpoints (.weights.h5) στον φάκελο ./savemodels")

best_model = files[files.size - 1]
print('best_model=', best_model)
model.load_weights(os.path.join(ckpt_dir, best_model))

In [ ]:
# --Confusion matrix ----------------------------------------------------------
print('TEST DATA-Confusion matrix:')
pred = model.predict(test_data)
pred_y = pred.argmax(axis=-1)

cm = confusion_matrix(test_labelp.astype('int32'), pred_y)
print(cm)

print('Accuracy ratios for each class')
print('Class 0 =', cm[0, 0] / np.sum(cm[0, :]))
print('Class 1 =', cm[1, 1] / np.sum(cm[1, :]))
print('Class 2 =', cm[2, 2] / np.sum(cm[2, :]))

In [ ]:
# -- Confusion matrix plot
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# labels από τα πραγματικά labels του test set (0/1/2)
labels_disp = np.array([str(c) for c in sorted(np.unique(test_labelp.astype('int32')))])

cmo = ConfusionMatrixDisplay(cm, display_labels=labels_disp)
fig, ax = plt.subplots(figsize=(8, 8))
cmo.plot(ax=ax, xticks_rotation=45)
plt.tight_layout()

In [ ]:
# Plot training and validation accuracy and loss graphs
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

np.save('historydata.npy', [acc, val_acc, loss, val_loss])

[acc, val_acc, loss, val_loss] = np.load('historydata.npy', allow_pickle=True)

plt.figure()
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r.', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r.', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()